In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from keras.models import Sequential, Model, model_from_json, load_model

Using TensorFlow backend.


In [2]:
## Load data URL
df = pd.read_csv('url_data.csv')
df.head(10) 

,url,isMalicious
0,songlyrics.com/news/riffd-the-shins-heartworms,0
1,imaging-resource.com/PRODS/olympus-e-m1-ii/oly...,0
2,gosugamers.net/lol/streams,0
3,thingiverse.com/corkyzett/collections/intlwome...,0
4,bausch.com/our-products/contact-lens-care/spec...,0
5,w88mobile.org/game/5151.html,0
6,datacenterdynamics.com/awards/latin-america-aw...,0
7,123people.com/s/marc+pageau,0
8,nownews.com/n/2017/03/21/2449327,0
9,wikipedia.org/wiki/Plague_Park,0


In [3]:
df['url'].head(10)

0       songlyrics.com/news/riffd-the-shins-heartworms
1    imaging-resource.com/PRODS/olympus-e-m1-ii/oly...
2                           gosugamers.net/lol/streams
3    thingiverse.com/corkyzett/collections/intlwome...
4    bausch.com/our-products/contact-lens-care/spec...
5                         w88mobile.org/game/5151.html
6    datacenterdynamics.com/awards/latin-america-aw...
7                          123people.com/s/marc+pageau
8                     nownews.com/n/2017/03/21/2449327
9                       wikipedia.org/wiki/Plague_Park
Name: url, dtype: object

In [4]:
#print([c for c in df['url'][1]])

In [5]:
#calculate the length of url
df['len']=df['url'].apply(lambda x:len([c for c in x]))
df[['url','len']].head()

,url,len
0,songlyrics.com/news/riffd-the-shins-heartworms,46
1,imaging-resource.com/PRODS/olympus-e-m1-ii/oly...,63
2,gosugamers.net/lol/streams,26
3,thingiverse.com/corkyzett/collections/intlwome...,51
4,bausch.com/our-products/contact-lens-care/spec...,70


In [6]:
#calculate the depth of url
def cal_depth(x):
    depth=0
    for i in [c for c in x]:
        if i=="/":
            depth+=1
        else:
            continue
    return depth

#make it into the dataframe
df['depth']=df['url'].apply(lambda x:cal_depth(x))
df[['url','depth']].head()

,url,depth
0,songlyrics.com/news/riffd-the-shins-heartworms,2
1,imaging-resource.com/PRODS/olympus-e-m1-ii/oly...,3
2,gosugamers.net/lol/streams,2
3,thingiverse.com/corkyzett/collections/intlwome...,3
4,bausch.com/our-products/contact-lens-care/spec...,3


In [7]:
#calculate the number of special character

SPEC_CHARS = ['+','\"','*','#','%','&','(',')','=','?','^','-','.','!','~','_','>','<']

def cal_spec_char(x):
    spec_char=0
    for i in [c for c in x]:
        if i in SPEC_CHARS:
            spec_char+=1
        else:
            continue
    return spec_char

df['spec_chars']=df['url'].apply(lambda x:cal_spec_char(x))
df[['url','spec_chars']].head()

,url,spec_chars
0,songlyrics.com/news/riffd-the-shins-heartworms,4
1,imaging-resource.com/PRODS/olympus-e-m1-ii/oly...,9
2,gosugamers.net/lol/streams,1
3,thingiverse.com/corkyzett/collections/intlwome...,1
4,bausch.com/our-products/contact-lens-care/spec...,7


In [8]:
#calculate number
NUMBERS = ['0','1','2','3','4','5','6','7','8','9']
def cal_num(x):
    num=0
    for i in [c for c in x]:
        if i in NUMBERS:
            num+=1
        else:
            continue
    return num
df['num_count']=df['url'].apply(lambda x:cal_num(x))
df[['url','num_count']].head()

,url,num_count
0,songlyrics.com/news/riffd-the-shins-heartworms,0
1,imaging-resource.com/PRODS/olympus-e-m1-ii/oly...,2
2,gosugamers.net/lol/streams,0
3,thingiverse.com/corkyzett/collections/intlwome...,0
4,bausch.com/our-products/contact-lens-care/spec...,0


In [14]:
#calculate the number of letter
#LETTERS = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','w','x','y','z']
df.head()

,url,isMalicious,len,depth,spec_chars,num_count
0,songlyrics.com/news/riffd-the-shins-heartworms,0,46,2,4,0
1,imaging-resource.com/PRODS/olympus-e-m1-ii/oly...,0,63,3,9,2
2,gosugamers.net/lol/streams,0,26,2,1,0
3,thingiverse.com/corkyzett/collections/intlwome...,0,51,3,1,0
4,bausch.com/our-products/contact-lens-care/spec...,0,70,3,7,0


In [15]:
#the features of url
url_feature=df[['len','depth','spec_chars','num_count','isMalicious']]
url_feature.head()

,len,depth,spec_chars,num_count,isMalicious
0,46,2,4,0,0
1,63,3,9,2,0
2,26,2,1,0,0
3,51,3,1,0,0
4,70,3,7,0,0


In [16]:
#in order to use autoencoder, we have to output the url labelled non-malicious
frauds = url_feature[url_feature.isMalicious==1]
normal = url_feature[url_feature.isMalicious==0]

In [19]:
from sklearn import model_selection

# use 0.8 of the data for training and the rest for testing
X_train, X_test = model_selection.train_test_split(url_feature, test_size=0.2)

In [21]:
# Take only the normal (non-malicious) URLs as training data
X_train = X_train[X_train.isMalicious == 0]
X_train = X_train.drop(['isMalicious'], axis=1)
y_test = X_test['isMalicious']
X_test = X_test.drop(['isMalicious'], axis=1)
X_train = X_train.values
X_test = X_test.values

In [24]:
#check the node for encoder
X_train.shape

(78052, 4)

In [25]:
#set input node
input_dim=4

In [28]:
from keras.models import Model 
from keras.layers import Dense, Input

#encoding dimension
encoding_dim=5

#autoencorder
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh")(input_layer)
encoder = Dense(int(encoding_dim), activation="relu")(encoder)
encoder = Dense(int(encoding_dim-2), activation="relu")(encoder)
code = Dense(int(encoding_dim-4), activation='tanh')(encoder)
decoder = Dense(int(encoding_dim-2), activation='tanh')(code)
decoder = Dense(int(encoding_dim), activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

In [ ]:
nb_epoch = 100
batch_size = 60
autoencoder.compile(optimizer='adam',
                    loss='mean_squared_error',
                    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)
                               
tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)
history = autoencoder.fit(X_train, X_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test, X_test),
                    verbose=1,
                    callbacks=[checkpointer, tensorboard]).history